In [ ]:
import pandas as pd 
from sklearn.model_selection import train_test_split
X_data=pd.read_csv("A:\\Coding\\ML\\Kaggle\\House-price-prediction-master\\train.csv",index_col='Id')
X_test=pd.read_csv("A:\\Coding\\ML\\Kaggle\\House-price-prediction-master\\test.csv",index_col='Id')
# SalePrice missing rows removal
X_data.dropna(axis=0,subset=['SalePrice'],inplace=True)

y=X_data.SalePrice
X_data.drop(['SalePrice'],axis=1,inplace=True)

X_train,X_valid,y_train,y_valid=train_test_split(X_data,y,train_size=0.8,test_size=0.2,random_state=1)

In [ ]:
cat_cols=[cname for cname in X_train.columns if X_train[cname].dtype=='object' and X_train[cname].nunique()<10]
num_cols=[cname for cname in X_train.columns if X_train[cname].dtype in ['int64','float64'] ]
my_cols=cat_cols+num_cols

X_train=X_train[my_cols]
X_valid=X_valid[my_cols]
X_test=X_test[my_cols]

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

numcol_transformer=SimpleImputer(strategy='mean')
catcol_transformer=Pipeline(steps=[('imputer',SimpleImputer(strategy='most_frequent')),
('onehot',OneHotEncoder(handle_unknown='ignore'))
])

In [6]:
from xgboost import XGBRegressor
preprocessor=ColumnTransformer(transformers=[('num_cols',numcol_transformer,num_cols),('cat_cols',catcol_transformer,cat_cols)])
model=XGBRegressor(n_estimators=500,random_state=0,learning_rate=.05)
mod_final=Pipeline(steps=[('preprocess',preprocessor),('model',model)])

mod_final.fit(X_train,y_train)

preds=mod_final.predict(X_valid)

abs_error=mean_absolute_error(y_valid,preds)
print(abs_error)

16063.137842465754
